In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.Collecting langchain==0.0.284 (from -r requirements.txt (line 1))
  Using cached langchain-0.0.284-py3-none-any.whl.metadata (14 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached streamlit-1.22.0-py2.py3-none-any.whl.metadata (7.3 kB)
  Using cached tiktoken-0.4.0.tar.gz (25 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'



ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement faiss-cpu==1.7.4 (from versions: 1.8.0, 1.8.0.post1, 1.9.0, 1.9.0.post1, 1.10.0, 1.11.0, 1.11.0.post1, 1.12.0, 1.13.0, 1.13.1)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-cpu==1.7.4


In [1]:
%pip install langchain-huggingface huggingface_hub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

from dotenv import load_dotenv
import os
load_dotenv("E:\\AI-Buildathon\\.env")
api_key = os.getenv("GOOGLE_GENAI_API_KEY")




In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, api_key=api_key)





In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdf_documents(file_path):
    loader= DirectoryLoader(
        file_path,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents=loader.load()
    return documents

document=load_pdf_documents("E:\\AI-Buildathon\\data")





In [5]:
for doc in document:
    doc.metadata.update({
        "source": "WHO",
        "category": "disease",
        "region": "Bangladesh",
        "language": "en",
        "confidence_level": "primary"
    })


In [6]:
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len,
    )
    text_chunk = text_splitter.split_documents(document)
    return text_chunk

text_chunk= text_splitter(document)
print(f"Number of Chunks: {len(text_chunk)}")

Number of Chunks: 20271


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")




In [8]:
load_dotenv("E:\\AI-Buildathon\\.env")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [9]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("test")

In [10]:
from langchain_pinecone import PineconeVectorStore

docstore = PineconeVectorStore.from_documents(
    documents=text_chunk,
    embedding=embedding_model,
    index_name="test",
)


In [11]:
docsearch= PineconeVectorStore.from_existing_index(
    embedding=embedding_model,
    index_name="test"
)

In [12]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import (create_stuff_documents_chain)

system_prompt=(
    "You are MaBondhu AI, a clinical assistant for maternal health. "
        "Provide safe, evidence-based, concise guidance. Use the following pieces of retrieved context to answer the question at the end. "
        "Language: {language}.\n\n"
        "Rules:\n"
        "1) If this is potentially life-threatening, say 'EMERGENCY' and recommend immediate referral.\n"
        "2) If unsure, recommend human-in-the-loop evaluation and list needed vitals/tests.\n"
        "3) Include 1–2 specific, actionable next steps (e.g., 'go to nearest clinic', 'take iron supplement 60mg').\n"
        "4) Provide concise Bangla/English text depending on language.\n\n"
        "5) If you don't know the answer, say 'Sorry, I don't know about it'.\n\n"
        "Answer now in clear {language} with citations (e.g., [WHO Section 3])."
        "\n\n"
        "Context:\n{context}\n\n"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [32]:
question_answering_chain = create_stuff_documents_chain(llm,prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)



In [40]:

inp=input("Enter your question: ")
response = rag_chain.invoke({
    "input": inp,
    "language": "English"
})

print(response['answer']) 

After the birth of your first child, focus on these key aspects:

1.  **Lead your baby's care:** Take charge of your baby's care, following the advice of doctors and midwives/birth attendants, and ensuring it works for your family. Never underestimate your opinion and ask all the questions you need to [Context 1, 2].
2.  **Involve family:** Involve other family members in your baby's care and support system [Context 1, 2].
3.  **Prioritize your wellbeing:** Seek information on lifestyle, working, and wellbeing to help you have a balanced and joyful experience [Context 3].

**Next Steps:**
*   Continue to consult with your doctors and midwives for personalized advice regarding your and your baby's health.
*   Actively communicate with family members to establish a supportive environment for your new family.
